In [6]:
import numpy as np

# Flip bit order in a byte (LSB <-> MSB)
def swap_bits(byte):
    return int(f"{byte:08b}"[::-1], 2)

# Whiten/dewhiten a list of bytes using a 7-bit LFSR.
def BLE_whitening(data, lfsr=0x01, polynomial=0x11):
    # Input data must be LSB first (0x7C = 124 must be formatted as 0011 1110)
    # LFSR default value is 0x01 as it is the default value in the nRF DATAWHITEIV register
    # The polynomial default value is 0x11 = 0b001_0001 -> x⁷ + x⁴ + 1 (x⁷ is omitted)
    output = np.empty_like(data)  # Initialise output array

    for idx, byte, in enumerate(data):
        whitened_byte = 0
        print(f'\n##### INDEX: {idx}, BYTE: {byte:08b} #####\n')
        for bit_pos in range(8):
            print(f'\nbit_pos: {bit_pos}')

            # XOR the current data bit with LFSR MSB
            lfsr_msb = (lfsr & 0x40) >> 6  # LFSR is 7-bit, so MSB is at position 6
            data_bit = (byte >> bit_pos) & 1  # Extract current bit
            whitened_bit = data_bit ^ lfsr_msb  # XOR
            print(f'data_bit: {data_bit}, lfsr_msb: {lfsr_msb}, whitened_bit: {whitened_bit}')
            
            # Update the whitened byte
            whitened_byte |= (whitened_bit << (bit_pos))
            print(f'whitened_byte: {whitened_byte:08b}')

            # Update LFSR
            if lfsr_msb:  # If MSB is 1 (before shifting), apply feedback
                lfsr = (lfsr << 1) ^ polynomial  # XOR with predefined polynomial
            else:
                lfsr <<= 1
            lfsr &= 0x7F  # 0x7F mask to keep ksfr within 7 bits

            print(f'lfsr: {lfsr:07b}')

        output[idx] = whitened_byte

    return output





# Example usage:
data = [0b00000010, 0b01010011, 0b01111101, 0b11100011, 0b10011000]  # Example received SDR data
data = [swap_bits(byte) for byte in data]
whitened = BLE_whitening(data)
print([f"0b{b:08b}" for b in whitened])  # Output in binary
print([f"{b}" for b in whitened])  # Output in decimal


##### INDEX: 0, BYTE: 01000000 #####


bit_pos: 0
data_bit: 0, lfsr_msb: 0, whitened_bit: 0
whitened_byte: 00000000
lfsr: 0000010

bit_pos: 1
data_bit: 0, lfsr_msb: 0, whitened_bit: 0
whitened_byte: 00000000
lfsr: 0000100

bit_pos: 2
data_bit: 0, lfsr_msb: 0, whitened_bit: 0
whitened_byte: 00000000
lfsr: 0001000

bit_pos: 3
data_bit: 0, lfsr_msb: 0, whitened_bit: 0
whitened_byte: 00000000
lfsr: 0010000

bit_pos: 4
data_bit: 0, lfsr_msb: 0, whitened_bit: 0
whitened_byte: 00000000
lfsr: 0100000

bit_pos: 5
data_bit: 0, lfsr_msb: 0, whitened_bit: 0
whitened_byte: 00000000
lfsr: 1000000

bit_pos: 6
data_bit: 1, lfsr_msb: 1, whitened_bit: 0
whitened_byte: 00000000
lfsr: 0010001

bit_pos: 7
data_bit: 0, lfsr_msb: 0, whitened_bit: 0
whitened_byte: 00000000
lfsr: 0100010

##### INDEX: 1, BYTE: 11001010 #####


bit_pos: 0
data_bit: 0, lfsr_msb: 0, whitened_bit: 0
whitened_byte: 00000000
lfsr: 1000100

bit_pos: 1
data_bit: 1, lfsr_msb: 1, whitened_bit: 0
whitened_byte: 00000000
lfsr: 0011001

b

In [7]:
import numpy as np

def binary_to_uint8_list(binary):
    # Ensure the binary array length is a multiple of 8
    if len(binary) % 8 != 0:
        raise ValueError("The binary list length must be a multiple of 8.")
    
    # Convert binary to NumPy array
    binary_array = np.array(binary, dtype=np.uint8)
    uint8_list = np.packbits(binary_array)

    # Swap bits in each byte (LSB first conversion)
    swapped_uint8_list = np.array([swap_bits(byte) for byte in uint8_list], dtype=np.uint8)

    return swapped_uint8_list

binary_data = np.array([0,0,0,0,0,0,1,0,  0,1,0,1,0,0,1,1], dtype=np.uint8)  # Two bytes: 0b00000010 and 0b01010011
uint8_result = binary_to_uint8_list(binary_data)

print(uint8_result)


[ 64 202]


In [8]:
def BLE_whitening(data, lfsr=0x01, polynomial=0x11):
    # Input data must be LSB first (0x7C = 124 must be formatted as 0011 1110)
    # LFSR default value is 0x01 as it is the default value in the nRF DATAWHITEIV register
    # The polynomial default value is 0x11 = 0b001_0001 -> x⁷ + x⁴ + 1 (x⁷ is omitted)
    output = np.empty_like(data)  # Initialise output array

    for idx, byte, in enumerate(data):
        whitened_byte = 0
        for bit_pos in range(8):
            # XOR the current data bit with LFSR MSB
            lfsr_msb = (lfsr & 0x40) >> 6  # LFSR is 7-bit, so MSB is at position 6
            data_bit = (byte >> bit_pos) & 1  # Extract current bit
            whitened_bit = data_bit ^ lfsr_msb  # XOR
            
            # Update the whitened byte
            whitened_byte |= (whitened_bit << (bit_pos))

            # Update LFSR
            if lfsr_msb:  # If MSB is 1 (before shifting), apply feedback
                lfsr = (lfsr << 1) ^ polynomial  # XOR with predefined polynomial
            else:
                lfsr <<= 1
            lfsr &= 0x7F  # 0x7F mask to keep ksfr within 7 bits

        output[idx] = whitened_byte

    return output, lfsr


# Example usage:
data = [0b00000010, 0b01010011, 0b01111101, 0b11100011, 0b10011000]  # Example received SDR data
data = [swap_bits(byte) for byte in data]
whitened, lsfr = BLE_whitening(data)
print([f"0b{b:08b}" for b in whitened])  # Output in binary
print([b for b in whitened])  # Output in decimal

['0b00000000', '0b01111000', '0b00000010', '0b00000100', '0b00000110']
[np.int64(0), np.int64(120), np.int64(2), np.int64(4), np.int64(6)]


In [3]:
import numpy as np

def compute_CRC(data, crc_init=0x00FFFF, crc_poly=0x00065B, crc_size=3):
    crc_mask = (1 << (crc_size * 8)) - 1  # Mask to n-byte width (0xFFFF for crc_size = 2)
    
    def swap_nbit(num, n):
        num = num & crc_mask
        reversed_bits = f'{{:0{n * 8}b}}'.format(num)[::-1]
        return int(reversed_bits, 2)
    
    crc_init = swap_nbit(crc_init, crc_size)  # LSB -> MSB
    crc_poly = swap_nbit(crc_poly, crc_size)
    
    crc = crc_init
    for byte in data:
        crc ^= int(byte)
        for _ in range(8):  # Process each bit
            if crc & 0x01:  # Check the LSB
                crc = (crc >> 1) ^ crc_poly
            else:
                crc >>= 1
            crc &= crc_mask  # Ensure CRC size
    
    return np.array([(crc >> (8 * i)) & 0xFF for i in range(crc_size)], dtype=np.uint8)


# Example usage (BLE)
data = np.array(
    [0x00, 0x78, 0x02, 0x04, 0x06, 0x08, 0x0A, 0x0C, 0x0E, 0x10, 0x12, 0x14, 0x16, 0x18, 0x1A, 0x1C, 
    0x1E, 0x20, 0x22, 0x24, 0x26, 0x28, 0x2A, 0x2C, 0x2E, 0x30, 0x32, 0x34, 0x36, 0x38, 0x3A, 0x3C, 
    0x3E, 0x40, 0x42, 0x44, 0x46, 0x48, 0x4A, 0x4C, 0x4E, 0x50, 0x52, 0x54, 0x56, 0x58, 0x5A, 0x5C, 
    0x5E, 0x60, 0x62, 0x64, 0x66, 0x68, 0x6A, 0x6C, 0x6E, 0x70, 0x72, 0x74, 0x76, 0x78, 0x7A, 0x7C, 
    0x7E, 0x80, 0x82, 0x84, 0x86, 0x88, 0x8A, 0x8C, 0x8E, 0x90, 0x92, 0x94, 0x96, 0x98, 0x9A, 0x9C, 
    0x9E, 0xA0, 0xA2, 0xA4, 0xA6, 0xA8, 0xAA, 0xAC, 0xAE, 0xB0, 0xB2, 0xB4, 0xB6, 0xB8, 0xBA, 0xBC, 
    0xBE, 0xC0, 0xC2, 0xC4, 0xC6, 0xC8, 0xCA, 0xCC, 0xCE, 0xD0, 0xD2, 0xD4, 0xD6, 0xD8, 0xDA, 0xDC, 
    0xDE, 0xE0, 0xE2, 0xE4, 0xE6, 0xE8, 0xEA, 0xEC, 0xEE, 0xF0], 
    dtype=np.uint8)


crc_result = compute_CRC(data)
print("CRC:", [f"0x{b:02X}" for b in crc_result])

# IEEE 802.15.4 example
data = np.array(
    [0x31, 0x32, 0x33], 
    dtype=np.uint8)

crc_result = compute_CRC(data, crc_init=0x0000, crc_poly=0x011021, crc_size=2)
print("CRC:", [f"0x{b:02X}" for b in crc_result])



CRC: ['0x82', '0x47', '0xC3']
CRC: ['0x78', '0x5A']
